In [47]:
import os
os.chdir("../")
os.getcwd()

'c:\\'

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
labels_df = pd.read_csv('C:\\Users\\agarc\\OneDrive\\Documentos\\GitHub\\Plant_Disease_Detection_for_ML\\artifacts\\workData\\df.csv')

# Combine the binary labels into a single class label
labels_df['class'] = labels_df[['healthy', 'multiple_diseases', 'rust', 'scab']].idxmax(axis=1)

# Update image paths to include ".jpg"
labels_df['image_id'] = labels_df['image_id'].apply(lambda x: f"{x}.jpg")

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['class'])


In [49]:
train_df

,Unnamed: 0,image_id,healthy,multiple_diseases,rust,scab,class
278,278,img_278.jpg,1,0,0,0,healthy
1601,1601,img_1601.jpg,1,0,0,0,healthy
8,8,img_8.jpg,0,0,0,1,scab
1986,1986,img_1986.jpg,0,1,0,0,multiple_diseases
2241,2241,img_2241.jpg,0,1,0,0,multiple_diseases
...,...,...,...,...,...,...,...
389,389,img_389.jpg,1,0,0,0,healthy
948,948,img_948.jpg,1,0,0,0,healthy
352,352,img_352.jpg,0,0,0,1,scab
627,627,img_627.jpg,0,0,1,0,rust


In [50]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
image_dir = 'C:\\Users\\agarc\\OneDrive\\Documentos\\GitHub\\Plant_Disease_Detection_for_ML\\artifacts\\preprocesamiento'  # Ensure this is the correct path

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_dir,
    x_col='image_id',  # Column with image filenames
    y_col='class',     # Column with class labels
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_dir,
    x_col='image_id',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1990 validated image filenames belonging to 4 classes.
Found 498 validated image filenames belonging to 4 classes.


In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=10,
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\agarc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/50


c:\Users\agarc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.2634 - loss: 1.7544 - val_accuracy: 0.3125 - val_loss: 1.3445
Epoch 2/50
 1/62 ━━━━━━━━━━━━━━━━━━━━ 1:37 2s/step - accuracy: 0.3750 - loss: 1.2980

c:\Users\agarc\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3750 - loss: 1.2980 - val_accuracy: 0.3889 - val_loss: 1.3536
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.3587 - loss: 1.3353 - val_accuracy: 0.3729 - val_loss: 1.3156
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3750 - loss: 1.2938 - val_accuracy: 0.3889 - val_loss: 1.2717
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.4660 - loss: 1.2087 - val_accuracy: 0.4854 - val_loss: 1.1875
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5000 - loss: 1.2724 - val_accuracy: 0.4444 - val_loss: 1.0803
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.5956 - loss: 0.9919 - val_accuracy: 0.6396 - val_loss: 0.9557
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7188 - loss: 0.7215 - val_accuracy: 0.7222 - val_loss: 0.9669
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.7470 - loss: 0.6795 - val_accuracy: 0.6646 - val_loss: 0.9375

In [ ]:
val_loss, val_acc = model.evaluate(val_generator, verbose=2)
print(f'Validation accuracy: {val_acc}')
print(f"validacion loss:{val_loss}")

16/16 - 5s - 329ms/step - accuracy: 0.6847 - loss: 0.9136
Validation accuracy: 0.6847389340400696
